In [ ]:
#################################################################################
# Laden aller relevate Module
#################################################################################
import pyspark
from pyspark.sql.functions import *
import json
import csv
from datetime import datetime
from delta import *
import delta

# use 95% of the screen for jupyter cell
from IPython.core.display import display, HTML
display(HTML("<style>.container {width:100% !important; }<style>"))

In [ ]:
# first for local usage pip install delta-spark

builder = pyspark.sql.SparkSession.builder.appName("MyApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars", "/Users/alor/opt/spark/jars/spark-sql-kafka-0-10_2.12-3.3.1.jar, /Users/alor/opt/spark/jars/kafka-clients-3.3.1.jar, /Users/alor/opt/spark/jars/spark-avro_2.12-3.3.1.jar") \
    .config("spark.driver.extraClassPath","/Users/alor/opt/spark/jars/spark-sql-kafka-0-10_2.12-3.3.1.jar, /Users/alor/opt/spark/jars/kafka-clients-3.3.1.jar, /Users/alor/opt/spark/jars/spark-avro_2.12-3.3.1.jar") \
    .config("spark.executor.extraClassPath","/Users/alor/opt/spark/jars/spark-sql-kafka-0-10_2.12-3.3.1.jar, /Users/alor/opt/spark/jars/kafka-clients-3.3.1.jar, /Users/alor/opt/spark/jars/spark-avro_2.12-3.3.1.jar")


spark = configure_spark_with_delta_pip(builder).getOrCreate()

sc=spark.sparkContext


In [ ]:
spark

# Bespiel 1: Tabelle Drop, Filter, Sort

## Dataframe - Drop, Filter, Sort

In [ ]:
df=(spark
    .read
    .format("csv")
    .option("header",True)
    .option("delimiter",";")
    .load("../2_lab/data/simple/")
   )

df.show(10)

In [ ]:
df2=(df
     .drop("date")
     .where(col("anzahl")>40)
     .sort(col("name").desc())
    )

df2.show(5)

## RDD - Drop, Filter, Sort

In [ ]:
rdd=sc.textFile("../2_lab/data/simple/")

# Zeilen von Strings
rdd.take(10)

In [ ]:
header=rdd.first()
rdd2=(rdd
      # remove first line
      .filter(lambda line: line != header )
      # split string into array
      .map(lambda line: line.split(";"))
      # change second element in array to int, drittes Element entfernen
      .map(lambda x: [x[0],int(x[1])])
      # filter on second array element (anzahl) < 40
      .filter(lambda x: x[1]>40)
      # sortby first array element descending
      .sortBy(lambda x: x[0],False)
     )
rdd2.take(5)

In [ ]:
rdd2.toDF(["name","anzahl"]).show(5)

# Beispiel 2: Word Count Fließtext

## RDD - reduce by key

In [ ]:
rdd4=sc.textFile("../2_lab/data/word-count/es-ist-nacht.txt")

# Zeilen von Strings
rdd4.take(10)

In [ ]:
rdd5=(rdd4
      # splitte alles in Einzelworte und einen langen Datensatz
      .flatMap(lambda line: line.split(" "))
      # mape jedes Wort auf ein Tupel mit 1
      .map(lambda word: (word,1))
      # alles über Keys addieren/reduzieren
      .reduceByKey(lambda x,y: x+y)
      # über key sortieren
      .sortByKey(False)
      # über zweite Spalte sortieren
      .sortBy(lambda x: x[1],False)
)

rdd5.take(10)

### Dataframe explode und groupby

In [ ]:
df4=(spark
    .read
    .format("text")
    .load("../2_lab/data/word-count/es-ist-nacht.txt")
   )

df4.show(10, truncate=False)

In [ ]:
df5=(df4
     # zeile in array splitten
     .withColumn("words",split(col("value")," "))
     # array in zeile exploden
     .withColumn("word",explode(col("words")))
     # nach gleichen Wörtern gruppieren und addieren
     .groupBy("word")
     .count()
     # sortieren nach count und word
     .sort(col("count").desc(),col("word").desc())
    )

df5.show(100,truncate=False)